In [1]:
import pandas as pd
import numpy as np
from icecream import ic

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

from tqdm import tqdm
tqdm.pandas()

import geopandas as gpd
import shapely

%load_ext autoreload
%autoreload 2

%config Completer.use_jedi = False

In [2]:
# each row is the intersection seen by a specific decoder and the number of cameras (and types/attached locations) in this intersection
all_counts = pd.read_csv("data/counts.csv")
# each row is the intersections over different areas of NY and its related info e.g.panorama id or whether there is a Traffic Signal present
all_intersections = pd.read_csv("data/intersections.csv")
# each row is the individual camera seen by a decoder at an intersection with its location/type i.e. building or PTZ/Dome
all_cameras = pd.read_csv("data/cameras.csv")

# Merge all intersections sharing the same PanoramaId

Some intersections share the same Panorama:

In [3]:
all_intersections.shape[0]

45021

In [4]:
all_intersections.drop_duplicates().shape[0]

45021

In [5]:
all_intersections[["PanoramaId"]].drop_duplicates().shape[0]

43668

And three panoramas appear in more than one Borough:

In [6]:
all_intersections[["PanoramaId", "Borough"]].drop_duplicates().shape[0]

43671

In [7]:
(
    all_intersections[["PanoramaId", "Borough"]].drop_duplicates().shape[0]
    - all_intersections[["PanoramaId"]].drop_duplicates().shape[0]
)

3

Luckily, PanoramaId and ImageDate are uniquely linked:

In [8]:
pano_date = all_intersections[["PanoramaId", "ImageDate"]].drop_duplicates()
pano_date.shape[0]

43668

## Drop "IntersectionId" to force using PanoramaId

In [9]:
all_intersections.head()

,IntersectionId,Url,Borough,TrafficSignal,Lat,Long,PanoramaId,ImageDate
0,AXjK_--KLVwDtjXR1esd,2238,The Bronx,NaN,40.890191,-73.819536,X7F7t3o1eR1gPT_3Mdhuiw,2019-11
1,AXjK_--dgxjkLnQCdqZ9,1704,The Bronx,NaN,40.859703,-73.845482,SEqkoCI1A4R3J6Q6OUiQDg,2012-04
2,AXjK_--uLVwDtjXR1ese,2239,The Bronx,NaN,40.856563,-73.869407,bebwdVkWcKQRIrqFnHQodQ,2019-10
3,AXjK_-09LVwDtjXR1esP,1689,The Bronx,NaN,40.859583,-73.866528,EkUAZd0irLLB24j_pqjjLA,2019-11
4,AXjK_-0aLVwDtjXR1esO,1688,The Bronx,TrafficLight,40.859675,-73.859799,kQTtiuw_dBjZuFsL7WsVvw,2020-10


In [10]:
all_counts.head()

,SubmissionId,DecoderId,DecoderGenericId,IntersectionId,StartTime,EndTime,n_cameras,attached_street,attached_building,attached_unknown,type_dome,type_bullet,type_unknown
0,camera_mappingHIVEcamera_mapping-camera_mappin...,AXj1zhMl2iuwklCl-p0m,686.0,AXjK_--KLVwDtjXR1esd,2021-04-24T20:45:38.297Z,2021-04-24T20:46:24Z,0,0,0,0,0,0,0
1,camera_mappingHIVEcamera_mapping-camera_mappin...,AXjRSPotLVwDtjXR1fRD,38022.0,AXjK_--KLVwDtjXR1esd,2021-04-22T20:56:58.527Z,2021-04-22T20:57:06Z,0,0,0,0,0,0,0
2,camera_mappingHIVEcamera_mapping-camera_mappin...,AXjTZXDggxjkLnQCdq55,37452.0,AXjK_--KLVwDtjXR1esd,2021-04-17T23:20:06.921Z,2021-04-17T23:20:57Z,0,0,0,0,0,0,0
3,camera_mappingHIVEcamera_mapping-camera_mappin...,AXkC8k-LLVwDtjXR1fjo,38728.0,AXjK_--dgxjkLnQCdqZ9,2021-04-27T03:16:03.606Z,2021-04-27T03:16:46Z,0,0,0,0,0,0,0
4,camera_mappingHIVEcamera_mapping-camera_mappin...,AXkG2KnD2iuwklCl-p8g,12915.0,AXjK_--dgxjkLnQCdqZ9,2021-04-25T02:52:52.877Z,2021-04-25T02:53:46Z,0,0,0,0,0,0,0


In [11]:
inter_to_pano = all_intersections[["IntersectionId", "PanoramaId"]].copy()
merged = all_counts.merge(inter_to_pano, on="IntersectionId", how="left")

Let's check that we have the same number of rows after merging.

In [12]:
ic(all_counts.shape)
ic(merged.shape)
assert all_counts.shape[0] == merged.shape[0]

ic| all_counts.shape: (137325, 13)
ic| merged.shape: (137325, 14)


Drop extra details, and **only keep PanoramaId** to merge intersections sharing a PanoramaId.

In [13]:
merged.head()

,SubmissionId,DecoderId,DecoderGenericId,IntersectionId,StartTime,EndTime,n_cameras,attached_street,attached_building,attached_unknown,type_dome,type_bullet,type_unknown,PanoramaId
0,camera_mappingHIVEcamera_mapping-camera_mappin...,AXj1zhMl2iuwklCl-p0m,686.0,AXjK_--KLVwDtjXR1esd,2021-04-24T20:45:38.297Z,2021-04-24T20:46:24Z,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
1,camera_mappingHIVEcamera_mapping-camera_mappin...,AXjRSPotLVwDtjXR1fRD,38022.0,AXjK_--KLVwDtjXR1esd,2021-04-22T20:56:58.527Z,2021-04-22T20:57:06Z,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
2,camera_mappingHIVEcamera_mapping-camera_mappin...,AXjTZXDggxjkLnQCdq55,37452.0,AXjK_--KLVwDtjXR1esd,2021-04-17T23:20:06.921Z,2021-04-17T23:20:57Z,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
3,camera_mappingHIVEcamera_mapping-camera_mappin...,AXkC8k-LLVwDtjXR1fjo,38728.0,AXjK_--dgxjkLnQCdqZ9,2021-04-27T03:16:03.606Z,2021-04-27T03:16:46Z,0,0,0,0,0,0,0,SEqkoCI1A4R3J6Q6OUiQDg
4,camera_mappingHIVEcamera_mapping-camera_mappin...,AXkG2KnD2iuwklCl-p8g,12915.0,AXjK_--dgxjkLnQCdqZ9,2021-04-25T02:52:52.877Z,2021-04-25T02:53:46Z,0,0,0,0,0,0,0,SEqkoCI1A4R3J6Q6OUiQDg


In [14]:
merged.drop(
    columns=[
        "SubmissionId",
        "DecoderId",
        "DecoderGenericId",
        "IntersectionId",
        "StartTime",
        "EndTime",
    ],
    inplace=True,
)

In [15]:
merged.head()

,n_cameras,attached_street,attached_building,attached_unknown,type_dome,type_bullet,type_unknown,PanoramaId
0,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
1,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
2,0,0,0,0,0,0,0,X7F7t3o1eR1gPT_3Mdhuiw
3,0,0,0,0,0,0,0,SEqkoCI1A4R3J6Q6OUiQDg
4,0,0,0,0,0,0,0,SEqkoCI1A4R3J6Q6OUiQDg


In [16]:
merged = merged.set_index(["PanoramaId"])

# Keep exactly 3 decoders per intersection

Count the number of decoders per intersection

In [17]:
merged.columns

Index(['n_cameras', 'attached_street', 'attached_building', 'attached_unknown',
       'type_dome', 'type_bullet', 'type_unknown'],
      dtype='object')

In [18]:
merged.head()

,n_cameras,attached_street,attached_building,attached_unknown,type_dome,type_bullet,type_unknown
PanoramaId,,,,,,,
X7F7t3o1eR1gPT_3Mdhuiw,0,0,0,0,0,0,0
X7F7t3o1eR1gPT_3Mdhuiw,0,0,0,0,0,0,0
X7F7t3o1eR1gPT_3Mdhuiw,0,0,0,0,0,0,0
SEqkoCI1A4R3J6Q6OUiQDg,0,0,0,0,0,0,0
SEqkoCI1A4R3J6Q6OUiQDg,0,0,0,0,0,0,0


In [19]:
pano_grouped = merged.groupby("PanoramaId")
three_or_more = pano_grouped.size() >= 3
kept_pano = three_or_more[three_or_more].index

In [20]:
n_less_than_3 = sum(pano_grouped.size() < 3)
ic(n_less_than_3)
x = merged.loc[kept_pano]
ic(merged.index.nunique())
ic(x.index.nunique())
assert x.index.nunique() == merged.index.nunique() - n_less_than_3

ic| n_less_than_3: 262
ic| merged.index.nunique(): 43668
ic| x.index.nunique(): 43406


In [21]:
kept_3 = x.groupby(level="PanoramaId").head(3)

In [22]:
kept_3.shape

(130218, 7)

In [23]:
merged.shape

(137325, 7)

In [24]:
stats = kept_3.stack().groupby(level=[0, 1]).agg(["median", "nunique"])

In [25]:
assert(all(stats["median"] % 1 == 0))

In [26]:
stats = stats.astype(int)

In [27]:
stats.head(n=10)

median  nunique
PanoramaId                                               
--48RnFczF55WcDFpy5y_Q n_cameras               0        2
                       attached_street         0        1
                       attached_building       0        2
                       attached_unknown        0        1
                       type_dome               0        1
                       type_bullet             0        1
                       type_unknown            0        1
--T2iuvfxXJrzbgD7R3W-g n_cameras               0        2
                       attached_street         0        1
                       attached_building       0        2

# Clarify n_unique column

In [28]:
description_nunique = {0: "IMPOSSIBLE", 1: "Unanimous", 2: "2 vs 1", 3: "All disagree"}
stats["agreement"] = stats["nunique"].apply(lambda n: description_nunique[n])
stats.head()

median  nunique  agreement
PanoramaId                                                          
--48RnFczF55WcDFpy5y_Q n_cameras               0        2     2 vs 1
                       attached_street         0        1  Unanimous
                       attached_building       0        2     2 vs 1
                       attached_unknown        0        1  Unanimous
                       type_dome               0        1  Unanimous

In [29]:
stats.drop("nunique", axis=1, inplace=True)

In [30]:
stats.unstack()

median                                    \
                       n_cameras attached_street attached_building   
PanoramaId                                                           
--48RnFczF55WcDFpy5y_Q         0               0                 0   
--T2iuvfxXJrzbgD7R3W-g         0               0                 0   
--VGAsqPgTXxgZFXuhuhbg         0               0                 0   
--YJY0GGvay_3W_O5ObeNQ         0               0                 0   
--fUkaf5nn1-bm4KiJ9pVA         1               0                 1   
...                          ...             ...               ...   
zzj23G_f-_hZa1LrXJ7tig         0               0                 0   
zzpZoM4JHrGNywYitkYT8w         1               0                 0   
zztPjLBFQjwEISsA75a-zg         1               1                 0   
zzv84J0v5tQobN5uCXO1xw         0               0                 0   
zzyhPWndbm7wxqTN6LuYLA         0               0                 0   

                                                                            \
                       attached_unknown type_dome type_bullet type_unknown   
PanoramaId                                                                   
--48RnFczF55WcDFpy5y_Q                0         0           0            0   
--T2iuvfxXJrzbgD7R3W-g                0         0           0            0   
--VGAsqPgTXxgZFXuhuhbg                0         0           0            0   
--YJY0GGvay_3W_O5ObeNQ                0         0           0            0   
--fUkaf5nn1-bm4KiJ9pVA                0         0           0            0   
...                                 ...       ...         ...          ...   
zzj23G_f-_hZa1LrXJ7tig                0         0           0            0   
zzpZoM4JHrGNywYitkYT8w                0         0           0            0   
zztPjLBFQjwEISsA75a-zg                0         1           0            0   
zzv84J0v5tQobN5uCXO1xw                0         0           0            0   
zzyhPWndbm7wxqTN6LuYLA                0         0           0            0   

                           agreement                                    \
                           n_cameras attached_street attached_building   
PanoramaId                                                               
--48RnFczF55WcDFpy5y_Q        2 vs 1       Unanimous            2 vs 1   
--T2iuvfxXJrzbgD7R3W-g        2 vs 1       Unanimous            2 vs 1   
--VGAsqPgTXxgZFXuhuhbg     Unanimous       Unanimous         Unanimous   
--YJY0GGvay_3W_O5ObeNQ        2 vs 1       Unanimous            2 vs 1   
--fUkaf5nn1-bm4KiJ9pVA        2 vs 1       Unanimous            2 vs 1   
...                              ...             ...               ...   
zzj23G_f-_hZa1LrXJ7tig        2 vs 1          2 vs 1         Unanimous   
zzpZoM4JHrGNywYitkYT8w        2 vs 1          2 vs 1            2 vs 1   
zztPjLBFQjwEISsA75a-zg  All disagree    All disagree            2 vs 1   
zzv84J0v5tQobN5uCXO1xw        2 vs 1       Unanimous            2 vs 1   
zzyhPWndbm7wxqTN6LuYLA        2 vs 1       Unanimous            2 vs 1   

                                                                                
                       attached_unknown     type_dome type_bullet type_unknown  
PanoramaId                                                                      
--48RnFczF55WcDFpy5y_Q        Unanimous     Unanimous   Unanimous    Unanimous  
--T2iuvfxXJrzbgD7R3W-g           2 vs 1     Unanimous   Unanimous    Unanimous  
--VGAsqPgTXxgZFXuhuhbg        Unanimous     Unanimous   Unanimous    Unanimous  
--YJY0GGvay_3W_O5ObeNQ        Unanimous     Unanimous   Unanimous    Unanimous  
--fUkaf5nn1-bm4KiJ9pVA        Unanimous     Unanimous   Unanimous    Unanimous  
...                                 ...           ...         ...          ...  
zzj23G_f-_hZa1LrXJ7tig        Unanimous     Unanimous   Unanimous       2 vs 1  
zzpZoM4JHrGNywYitkYT8w        Unanimous        2 vs 1   Unanimous    Unanimous  
zztPjLBFQjwEISsA75a-zg  

In [31]:
stats_wide = stats.unstack()

In [32]:
stats_wide.columns = ["_".join(reversed(col)) for col in stats_wide.columns.values]

In [33]:
stats_wide.head()

,n_cameras_median,attached_street_median,attached_building_median,attached_unknown_median,type_dome_median,type_bullet_median,type_unknown_median,n_cameras_agreement,attached_street_agreement,attached_building_agreement,attached_unknown_agreement,type_dome_agreement,type_bullet_agreement,type_unknown_agreement
PanoramaId,,,,,,,,,,,,,,
--48RnFczF55WcDFpy5y_Q,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous
--T2iuvfxXJrzbgD7R3W-g,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,2 vs 1,Unanimous,Unanimous,Unanimous
--VGAsqPgTXxgZFXuhuhbg,0,0,0,0,0,0,0,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous
--YJY0GGvay_3W_O5ObeNQ,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous
--fUkaf5nn1-bm4KiJ9pVA,1,0,1,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous


In [34]:
stats_wide.columns

Index(['n_cameras_median', 'attached_street_median',
       'attached_building_median', 'attached_unknown_median',
       'type_dome_median', 'type_bullet_median', 'type_unknown_median',
       'n_cameras_agreement', 'attached_street_agreement',
       'attached_building_agreement', 'attached_unknown_agreement',
       'type_dome_agreement', 'type_bullet_agreement',
       'type_unknown_agreement'],
      dtype='object')

# Merge with Lat/Lon for each Panorama, as obtained from Google

## Load panoramas and remove duplicates

In [35]:
google_panoramas = pd.read_csv("data/panorama_url.csv")

In [36]:
google_panoramas.head()

,PanoramaId,Lat,Long,GoogleLat,GoogleLong
0,X7F7t3o1eR1gPT_3Mdhuiw,40.890191,-73.819536,40.890166,-73.819454
1,SEqkoCI1A4R3J6Q6OUiQDg,40.859703,-73.845482,40.859723,-73.845604
2,bebwdVkWcKQRIrqFnHQodQ,40.856563,-73.869407,40.856624,-73.869245
3,EkUAZd0irLLB24j_pqjjLA,40.859583,-73.866528,40.859556,-73.866448
4,kQTtiuw_dBjZuFsL7WsVvw,40.859675,-73.859799,40.859676,-73.859766


In [37]:
google_panoramas.drop(["Lat", "Long"], axis=1, inplace=True)
google_panoramas.rename(
    columns={"GoogleLat": "Lat", "GoogleLong": "Long"}, inplace=True
)

In [38]:
ic(google_panoramas.shape[0])
google_panoramas.drop_duplicates(inplace=True)
ic(google_panoramas.shape[0])

ic| google_panoramas.shape[0]: 45021
ic| google_panoramas.shape[0]: 43668


43668

In [39]:
google_panoramas.head()

,PanoramaId,Lat,Long
0,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454
1,SEqkoCI1A4R3J6Q6OUiQDg,40.859723,-73.845604
2,bebwdVkWcKQRIrqFnHQodQ,40.856624,-73.869245
3,EkUAZd0irLLB24j_pqjjLA,40.859556,-73.866448
4,kQTtiuw_dBjZuFsL7WsVvw,40.859676,-73.859766


# Add Borough

## Obtain borough shapes from NYC.gov

We obtain the borough boundaries including water from NYC.gov website: https://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page

In [40]:
boroughs = gpd.read_file(
    "data/nyc_borough_boundary_water_query.json"
)

In [41]:
boroughs = boroughs[["BoroName", "geometry"]]
boroughs.head()

,BoroName,geometry
0,Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ..."
1,Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901..."
2,Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724..."
3,Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639..."
4,Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793..."


## Check if we have any overlap accross boroughs

In [42]:
cross = boroughs.merge(boroughs, how="cross")

In [43]:
cross.columns

Index(['BoroName_x', 'geometry_x', 'BoroName_y', 'geometry_y'], dtype='object')

In [44]:
cross["overlap"] = cross.apply(
    lambda row: row["geometry_x"].overlaps(row["geometry_y"]), axis=1
)

In [45]:
cross[(cross["overlap"] == True) & (cross["BoroName_x"] != cross["BoroName_y"])]

,BoroName_x,geometry_x,BoroName_y,geometry_y,overlap
1,Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ...",Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901...",True
2,Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ...",Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724...",True
4,Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ...",Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793...",True
5,Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901...",Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ...",True
9,Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901...",Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793...",True
10,Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724...",Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ...",True
13,Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724...",Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639...",True
14,Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724...",Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793...",True
17,Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639...",Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724...",True
19,Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639...",Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793...",True


## Check panorama across each Borough

Creating a Point geometry for each Panorama

In [46]:
geo_panos = gpd.GeoDataFrame(
    google_panoramas,
    geometry=gpd.points_from_xy(google_panoramas.Long, google_panoramas.Lat),
)

Cross product to compare all panos to all boroughs.

In [47]:
pano_cross_borough = geo_panos.merge(
    boroughs, how="cross", suffixes=["_pano", "_borough"]
)

In [48]:
pano_cross_borough

,PanoramaId,Lat,Long,geometry_pano,BoroName,geometry_borough
0,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ..."
1,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901..."
2,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724..."
3,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639..."
4,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Queens,"POLYGON ((-73.77896 40.81172, -73.76371 40.793..."
...,...,...,...,...,...,...
218335,1qZpfGIN_Rh6NCN8qa70Lg,40.637582,-74.133613,POINT (-74.13361 40.63758),Manhattan,"MULTIPOLYGON (((-74.04388 40.69020, -74.04351 ..."
218336,1qZpfGIN_Rh6NCN8qa70Lg,40.637582,-74.133613,POINT (-74.13361 40.63758),Bronx,"POLYGON ((-73.86477 40.90202, -73.86305 40.901..."
218337,1qZpfGIN_Rh6NCN8qa70Lg,40.637582,-74.133613,POINT (-74.13361 40.63758),Brooklyn,"POLYGON ((-73.92723 40.72534, -73.92654 40.724..."
218338,1qZpfGIN_Rh6NCN8qa70Lg,40.637582,-74.133613,POINT (-74.13361 40.63758),Staten Island,"POLYGON ((-74.05582 40.64972, -74.05620 40.639..."


Compute intersections

In [49]:
pano_cross_borough["within"] = pano_cross_borough.progress_apply(
    lambda row: row["geometry_borough"].contains(row["geometry_pano"]), axis=1
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 218340/218340 [00:03<00:00, 54841.66it/s]


Do we have panoramas with more than one borough, or with no borough?

In [50]:
n_borough_per_pano = pano_cross_borough.groupby("PanoramaId").agg(
    n_borough=("within", sum)
)
panos_without_borough = n_borough_per_pano[n_borough_per_pano["n_borough"] != 1].copy()
panos_without_borough["BoroName"] = np.NaN
panos_without_borough.drop(columns="n_borough", inplace=True)
panos_without_borough.reset_index(inplace=True)
panos_without_borough

,PanoramaId,BoroName
0,-Sigf-Bv6ijXTHUGMg2zxg,NaN
1,9oPdYuT1g4NW3DyyUlkgBw,NaN
2,BS3ouvtr5BW3auudWk4dAQ,NaN
3,ByFmxX31h1WQrpVNDC05GA,NaN
4,EjE45e2qBJQlidBrBps16g,NaN
5,JI2z-W8_dGOjFuQUhtvvyQ,NaN
6,RT9hmK5PGZ9xOBTdhS2gwg,NaN
7,TWn86Fwc-fDs71ZBvwspRA,NaN
8,WhgmPp3q8h_W6a2WTsQriw,NaN
9,ZMwz7bzm2jhsFlKLaWbvYA,NaN


Let's keep just the right borough, then

In [51]:
pano_cross_borough.columns

Index(['PanoramaId', 'Lat', 'Long', 'geometry_pano', 'BoroName',
       'geometry_borough', 'within'],
      dtype='object')

In [52]:
panos_with_boroughs = pano_cross_borough[pano_cross_borough["within"] == True].copy()
panos_with_boroughs.drop(columns=["within", "geometry_borough"], inplace=True)

In [53]:
panos_with_boroughs.head()

,PanoramaId,Lat,Long,geometry_pano,BoroName
1,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Bronx
6,SEqkoCI1A4R3J6Q6OUiQDg,40.859723,-73.845604,POINT (-73.84560 40.85972),Bronx
11,bebwdVkWcKQRIrqFnHQodQ,40.856624,-73.869245,POINT (-73.86925 40.85662),Bronx
16,EkUAZd0irLLB24j_pqjjLA,40.859556,-73.866448,POINT (-73.86645 40.85956),Bronx
21,kQTtiuw_dBjZuFsL7WsVvw,40.859676,-73.859766,POINT (-73.85977 40.85968),Bronx


In [54]:
ic(panos_with_boroughs.shape[0])
ic(google_panoramas.shape[0])
assert (
    google_panoramas.shape[0]
    == panos_with_boroughs.shape[0] + panos_without_borough.shape[0]
)

ic| panos_with_boroughs.shape[0]: 43655
ic| google_panoramas.shape[0]: 43668


In [55]:
panos_boroughed = pd.concat([panos_with_boroughs, panos_without_borough])
panos_boroughed

,PanoramaId,Lat,Long,geometry_pano,BoroName
1,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Bronx
6,SEqkoCI1A4R3J6Q6OUiQDg,40.859723,-73.845604,POINT (-73.84560 40.85972),Bronx
11,bebwdVkWcKQRIrqFnHQodQ,40.856624,-73.869245,POINT (-73.86925 40.85662),Bronx
16,EkUAZd0irLLB24j_pqjjLA,40.859556,-73.866448,POINT (-73.86645 40.85956),Bronx
21,kQTtiuw_dBjZuFsL7WsVvw,40.859676,-73.859766,POINT (-73.85977 40.85968),Bronx
...,...,...,...,...,...
8,WhgmPp3q8h_W6a2WTsQriw,NaN,NaN,None,NaN
9,ZMwz7bzm2jhsFlKLaWbvYA,NaN,NaN,None,NaN
10,mEtqRWw3tC2f16opIvDSrw,NaN,NaN,None,NaN
11,rtaNkwhlqk3kqjOMjCZiEg,NaN,NaN,None,NaN


# Merge all and add URL and image date

In [56]:
panos_boroughed["URL"] = (
    "https://www.google.com/maps/@?api=1&map_action=pano&pano="
    + panos_boroughed["PanoramaId"]
)
panos_url_date = panos_boroughed.merge(pano_date, on="PanoramaId", how="left")

In [57]:
panos_url_date

,PanoramaId,Lat,Long,geometry_pano,BoroName,URL,ImageDate
0,X7F7t3o1eR1gPT_3Mdhuiw,40.890166,-73.819454,POINT (-73.81945 40.89017),Bronx,https://www.google.com/maps/@?api=1&map_action...,2019-11
1,SEqkoCI1A4R3J6Q6OUiQDg,40.859723,-73.845604,POINT (-73.84560 40.85972),Bronx,https://www.google.com/maps/@?api=1&map_action...,2012-04
2,bebwdVkWcKQRIrqFnHQodQ,40.856624,-73.869245,POINT (-73.86925 40.85662),Bronx,https://www.google.com/maps/@?api=1&map_action...,2019-10
3,EkUAZd0irLLB24j_pqjjLA,40.859556,-73.866448,POINT (-73.86645 40.85956),Bronx,https://www.google.com/maps/@?api=1&map_action...,2019-11
4,kQTtiuw_dBjZuFsL7WsVvw,40.859676,-73.859766,POINT (-73.85977 40.85968),Bronx,https://www.google.com/maps/@?api=1&map_action...,2020-10
...,...,...,...,...,...,...,...
43663,WhgmPp3q8h_W6a2WTsQriw,NaN,NaN,None,NaN,https://www.google.com/maps/@?api=1&map_action...,2018-11
43664,ZMwz7bzm2jhsFlKLaWbvYA,NaN,NaN,None,NaN,https://www.google.com/maps/@?api=1&map_action...,2018-07
43665,mEtqRWw3tC2f16opIvDSrw,NaN,NaN,None,NaN,https://www.google.com/maps/@?api=1&map_action...,2014-09
43666,rtaNkwhlqk3kqjOMjCZiEg,NaN,NaN,None,NaN,https://www.google.com/maps/@?api=1&map_action...,2020-11


In [58]:
panos_url_date.set_index("PanoramaId")
final_full_data = stats_wide.join(panos_url_date.set_index("PanoramaId"), how="left")

In [59]:
stats_wide.shape

(43406, 14)

# Save final dataset as csv

In [60]:
final_full_data

,n_cameras_median,attached_street_median,attached_building_median,attached_unknown_median,type_dome_median,type_bullet_median,type_unknown_median,n_cameras_agreement,attached_street_agreement,attached_building_agreement,attached_unknown_agreement,type_dome_agreement,type_bullet_agreement,type_unknown_agreement,Lat,Long,geometry_pano,BoroName,URL,ImageDate
PanoramaId,,,,,,,,,,,,,,,,,,,,
--48RnFczF55WcDFpy5y_Q,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous,40.870564,-73.869327,POINT (-73.86933 40.87056),Bronx,https://www.google.com/maps/@?api=1&map_action...,2019-08
--T2iuvfxXJrzbgD7R3W-g,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,2 vs 1,Unanimous,Unanimous,Unanimous,40.669099,-73.762447,POINT (-73.76245 40.66910),Queens,https://www.google.com/maps/@?api=1&map_action...,2020-10
--VGAsqPgTXxgZFXuhuhbg,0,0,0,0,0,0,0,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous,Unanimous,40.766790,-73.771211,POINT (-73.77121 40.76679),Queens,https://www.google.com/maps/@?api=1&map_action...,2020-11
--YJY0GGvay_3W_O5ObeNQ,0,0,0,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous,40.554253,-74.201603,POINT (-74.20160 40.55425),Staten Island,https://www.google.com/maps/@?api=1&map_action...,2019-09
--fUkaf5nn1-bm4KiJ9pVA,1,0,1,0,0,0,0,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous,40.611084,-74.162803,POINT (-74.16280 40.61108),Staten Island,https://www.google.com/maps/@?api=1&map_action...,2019-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzj23G_f-_hZa1LrXJ7tig,0,0,0,0,0,0,0,2 vs 1,2 vs 1,Unanimous,Unanimous,Unanimous,Unanimous,2 vs 1,40.889133,-73.821303,POINT (-73.82130 40.88913),Bronx,https://www.google.com/maps/@?api=1&map_action...,2019-11
zzpZoM4JHrGNywYitkYT8w,1,0,0,0,0,0,0,2 vs 1,2 vs 1,2 vs 1,Unanimous,2 vs 1,Unanimous,Unanimous,40.750994,-73.706451,POINT (-73.70645 40.75099),Queens,https://www.google.com/maps/@?api=1&map_action...,2019-07
zztPjLBFQjwEISsA75a-zg,1,1,0,0,1,0,0,All disagree,All disagree,2 vs 1,Unanimous,All disagree,2 vs 1,Unanimous,40.730852,-73.947876,POINT (-73.94788 40.73085),Brooklyn,https://www.google.com/maps/@?api=1&map_action...,2020-10


In [61]:
final_full_data.to_csv("data/counts_per_intersections.csv", mode="w")